In [4]:
import threading

lock = threading.Lock()

def recursive(n: int) -> int:
    lock.acquire()
    if n > 0:
        return n * recursive(n-1)
    else:
        return 1
    lock.release()

# 会卡住: 普通互斥锁threading.Lock, 当同一线程重复获取会死锁
# recursive(3)

In [6]:
rlock = threading.RLock()

def recursive_rl(n: int) -> int:
    with rlock:
        if n > 0:
            return n * recursive_rl(n-1)
        else:
            return 1

print(recursive_rl(3))

6


### threading.local
TLS: thrading local storage

In [ ]:
import threading
import time

class TSLData(threading.local):
    def __init__(self):
        self.value = 0


# 这个TSLData实例本身是全局所有线程共享的(所有线程访问同一个对象)
# 但是通过threading.local的魔法, 它的属性data.value是线程隔离的. TODO: threading.local是如何实现这一点的? 参考CPython: _threading_local.py. 代码不长
# 比喻: 想象一个共享的保险箱（data），但每个线程用自己的钥匙打开时，会看到独立的抽屉（data.value）。虽然保险箱是同一个，但抽屉里的内容互不可见。
data = TSLData()

def worker():
    data.value = threading.get_ident()
    time.sleep(3)
    print(f"Thread {threading.get_ident()}: value = {data.value}\n")


threads = []
for _ in range(3):
    t = threading.Thread(target=worker)
    threads.append(t)
    t.start()

start_time = time.time()
for t in threads:
    t.join()
print(f"It took {time.time() - start_time} s")

print(f"Main thread {threading.get_ident()}: value = {data.value}\n")

Thread 13105360896: value = 13105360896Thread 13122150400: value = 13122150400Thread 13138939904: value = 13138939904


It took 3.005528688430786 s
Main thread 8667832832: value = 0



## 自己实现thread.local
TLS对象在每个线程有一份自己的属性, 这里会涉及到几个magic methods:
1. \_\_getattribute\_\_(self, name): 访问对象的任何属性时触发
2. \_\_getattr\_\_(self, name):  当访问不存在的属性时触发（\_\_getattribute\_\_ 找不到属性后才会调用它）
3. \_\_setattr\_\_(self, name, value): 当设置对象的属性时触发
4. \_\_delattr\_\_(self, name): 当删除对象的属性时触发(del obj.x)

虽然不是魔术方法，但 \_\_dict\_\_ 存储了对象的所有属性（除非使用了\_\_slots\_\_）. 可以覆盖 \_\_getattribute\_\_ 和 \_\_setattr\_\_ 来模拟 \_\_dict\_\_ 的行为

In [121]:
class BadRecursionExample:
    def __init__(self):
        self.x = 10

    def __getattribute__(self, name):
        # 直接访问 self.name 会再次触发 __getattribute__，导致无限递归
        return self.__dict__[name]    # 这里会无限调用__getattribute__

obj = BadRecursionExample()
print(obj.x)

RecursionError: maximum recursion depth exceeded

In [ ]:
class GoodExample:
    def __init__(self):
        self.x = 10

    def __getattribute__(self, name):
        # 使用object.__getattribute__可以直接调用object基类的__getattribute__方法, 绕过当前类可能重写的__getattribute__
        return object.__getattribute__(self, name)

obj = GoodExample()
print(obj.x)

10


In [ ]:
import time
from weakref import ref
from contextlib import contextmanager

# 一个简单的thread local object实现, 完整的实现参考CPython _threading_local.py文件


@contextmanager
def _patch(self):
    impl = object.__getattribute__(self, "_local__impl")
    try:
        dct = impl.get_dict()
    except KeyError:
        dct = impl.create_dict()
        args, kw = impl.localargs
        self.__init__(*args, **kw)

    with impl.locallock:
        # 使用object.__getattribute__可以直接调用object基类的__getattribute__方法, 绕过当前类可能重写的__getattribute__
        print(f"before subtitution, __dict__ = {object.__getattribute__(self, '__dict__')}")
        object.__setattr__(self, "__dict__", dct)
        print(f"after subtitution, __dict__ = {dct}")

        yield

class _localimpl:
    def __init__(self):
        self.dicts = {}

    def get_dict(self):
        thread = threading.current_thread()
        return self.dicts[id(thread)]

    def create_dict(self):
        local_dict = {}
        thread = threading.current_thread()
        self.dicts[id(thread)] = local_dict
        return local_dict


class TLSData:
    __slots__ = '_local__impl', '__dict__'

    def __new__(cls, /, *args, **kwargs):
        # 保证TLSData在构造时不进行初始化, 每个thread单独进行初始化
        if (args or kwargs) and (cls.__init__ is object.__init__):
            raise TypeError("TSLData does not take arguments...")
        self = object.__new__(cls)
        impl = _localimpl()
        impl.localargs = (args, kwargs)
        impl.locallock = threading.RLock()
        object.__setattr__(self, "_local__impl", impl)
        impl.create_dict()
        return self


    def __getattribute__(self, name):
        with _patch(self):
            # print(f"Inside getattribute patch, name = {name}")
            return object.__getattribute__(self, name)

    def __setattr__(self, name, value):
        with _patch(self):
            # print(f"Inside setattr patch, name = {name}, value = {value}")
            return object.__setattr__(self, name, value)

    def __delattr__(self, name):
        with _patch(self):
            return object.__delattr__(self, name)


from threading import current_thread, RLock

tls = TLSData()

def worker():
    tls.value = threading.get_ident()
    print(f"In {threading.get_ident()}, tls.value = {tls.value}")


threads = []
for idx in range(3):
    t = threading.Thread(target=worker)
    threads.append(t)
    t.start()

for thread in threads:
    thread.join()

before subtitution, __dict__ = {}
after subtitution, __dict__ = {}
before subtitution, __dict__ = {}
after subtitution, __dict__ = {}
before subtitution, __dict__ = {}
after subtitution, __dict__ = {}
before subtitution, __dict__ = {'value': 13122150400}
after subtitution, __dict__ = {'value': 13122150400}
In 13122150400, tls.value = 13122150400
before subtitution, __dict__ = {'value': 13122150400}
after subtitution, __dict__ = {}
before subtitution, __dict__ = {'value': 13105360896}
after subtitution, __dict__ = {'value': 13105360896}
In 13105360896, tls.value = 13105360896
before subtitution, __dict__ = {'value': 13105360896}
after subtitution, __dict__ = {}
before subtitution, __dict__ = {}
after subtitution, __dict__ = {}
before subtitution, __dict__ = {'value': 13138939904}
after subtitution, __dict__ = {'value': 13138939904}
In 13138939904, tls.value = 13138939904
